In [1]:
import pandas as pd

Import the dataset as a DataFrame and storage it in the data variable

In [2]:
data = pd.read_csv('_src/data/movies_dataset.csv', encoding='UTF-8', delimiter=',', )

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13816\593806153.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('_src/data/movies_dataset.csv', encoding='UTF-8', delimiter=',', )


Import the credits dataset for later on extract the director name

In [3]:
credits = pd.read_csv('_src/data/credits.csv')
credits.drop(columns=['cast'], inplace=True)

Transforme id into a integer

In [4]:
try:
    data['id'].astype(int)
except ValueError as e:
    print(e)

invalid literal for int() with base 10: '1997-08-20'


We can see that the data is all wrong, so we drop it

In [5]:
data.drop(data.loc[data['id'].str.contains('-', regex=True)].index, inplace=True)

In [6]:
data['id'] = data['id'].astype(int)

Merge the dataframes so we work with only 1

In [7]:
data = pd.merge(data,credits, on='id')

Drop the innecesary columns

In [8]:
data.drop(columns=['adult','homepage','imdb_id','original_title','poster_path','video'], inplace = True)

### Extract the nested data that we need and create its dataframe's columns ### 

Nested data in 'belongs_to_collection'

In [9]:
#Crate the new columns of the DataFrame
data['collection_name'] = 0

#Asign to the new columns the values in belongs_to_collecition < id and name
for i in range(len(data['belongs_to_collection'])):
    if (len(str(data['belongs_to_collection'][i])) > 10):
        data['belongs_to_collection'][i] = data['belongs_to_collection'][i].split(',')
        data.loc[i, 'collection_name'] = data['belongs_to_collection'][i][1][10:-1]

#Drop belongs_to_collection
data.drop(columns = 'belongs_to_collection', inplace = True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13816\757821891.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['belongs_to_collection'][i] = data['belongs_to_collection'][i].split(',')


nested data in 'crew'

In [10]:
def extract_director(crew_str):    
    crew_list = crew_str.split('Director')
    if(len(crew_list) > 1):
        director = crew_list[1].split(',') 
        return director[1][10:-1]       

data['director'] = data['crew'].apply(extract_director)

data.drop(columns='crew', inplace = True)

Nested data in 'genres'

In [11]:
# Define a function to extract genre names from genres
def extract_genre_names(genres_str):
    if(len(str(genres_str)) > 4):
        genre_list = genres_str.split(',')
        genre_names = [element.strip(']')[10:-2] for element in genre_list if element[2:6] == 'name']
        return ','.join(genre_names)

# Apply the function to create the 'genres_list' column
data['genres_list'] = data['genres'].apply(extract_genre_names)

# Drop 'genres' column
data.drop(columns='genres', inplace=True)

Nested data in 'production_companies'

In [12]:
#Crate the new columns of the DataFrame
data['prod_companies'] = ''

def extract_comp_names(comp_str):
    if(len(str(comp_str)) > 4):
        genre_list = comp_str.split(',')
        comp_names = [element.strip(']')[11:-1] for element in genre_list if element[3:7] == 'name']
        return ','.join(comp_names)

# Apply the function to create the 'genres_list' column
data['prod_companies'] = data['production_companies'].apply(extract_comp_names)

# Drop 'genres' column
data.drop(columns='production_companies', inplace = True)

data['prod_companies'].fillna('None', inplace = True)

Nested data in 'production_countries'

In [13]:
data['prod_country'] = data['production_countries'].apply(lambda x: x.split(',')[1][10:-2].strip("'") if len(str(x)) > 4 else '')

data.drop(columns='production_countries', inplace=True)

Nested data in 'spoken_languages'

In [14]:
data['release_language'] = data['spoken_languages'].apply(lambda x: x.split(',')[1][10:-2].strip("'") if len(str(x)) > 4 else '')

data.drop(columns='spoken_languages', inplace=True)

In [15]:
data.head(8)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection_name,director,genres_list,prod_companies,prod_country,release_language
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,John Lasseter,"Animation,Comedy,Family",Pixar Animation Studios,United States of America,English
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,0,Joe Johnston,"Adventure,Fantasy,Family","TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,English
2,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,Howard Deutch,"Romance,Comedy","Warner Bros.,Lancaster Gate",United States of America,English
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,0,Forest Whitaker,"Comedy,Drama,Romance",Twentieth Century Fox Film Corporation,United States of America,English
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,Elliot Davis,Comedy,"Sandollar Productions,Touchstone Pictures",United States of America,English
5,60000000,949,en,"Obsessive master thief, Neil McCauley leads a ...",17.924927,1995-12-15,187436818.0,170.0,Released,A Los Angeles Crime Saga,Heat,7.7,1886.0,0,Michael Mann,"Action,Crime,Drama,Thriller","Regency Enterprises,Forward Pass,Warner Bros.",United States of America,English
6,58000000,11860,en,An ugly duckling having undergone a remarkable...,6.677277,1995-12-15,0.0,127.0,Released,You are cordially invited to the most surprisi...,Sabrina,6.2,141.0,0,Sydney Pollack,"Comedy,Romance","Paramount Pictures,Scott Rudin Productions,Mir...",Germany,Français
7,0,45325,en,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,1995-12-22,0.0,97.0,Released,The Original Bad Boys.,Tom and Huck,5.4,45.0,0,Peter Hewitt,"Action,Adventure,Drama,Family",Walt Disney Pictures,United States of America,English


Create a column 'return' with the return of investmen. (revenue / budget). In case that there is no data to operate it has to take the value '0' 

Fill the nulls values in budget and revenue with 0

In [16]:
fill_values = {'revenue': 0, 'budget': 0}
data.fillna(fill_values, inplace=True)

Now that we have all the data, we change the dtypes of the necesary columns to make the operations

In [17]:
int_columns = ['budget','revenue']
float_columns = ['popularity','vote_average']

data[int_columns] = data[int_columns].astype(int)
data[float_columns] = data[float_columns].astype(float)

c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


Create the 'return' column

In [18]:
data['return'] = round(data['revenue'] / data['budget'],2)

In [19]:
data.loc[data['return'] == float('inf'), 'return'] = data['revenue']

The dates has to have the format AAAA-mm-dd, and with that create the column release_year where we will extract the year of release

In [20]:
data['release_date'] = pd.to_datetime(data['release_date'])
data.dropna(subset=['release_date'], inplace=True)
data['release_year'] = data['release_date'].dt.year

In [21]:
new_columns = ['collection_name','director', 'genres_list', 'id', 
               'title', 'prod_companies', 'prod_country', 'original_language', 
               'release_language', 'overview', 'popularity', 'release_date','release_year', 'runtime', 
               'status', 'tagline', 'vote_average', 'vote_count', 'budget', 'revenue','return']
data = data.reindex(columns = new_columns)

In [22]:
data.loc[data['prod_companies'].isna(), 'title']

Series([], Name: title, dtype: object)

Drop the movies with no release-date

In [23]:
data.to_csv('_src/data/movies_normalized.csv', index=False)